In [2]:
import json
import time

from tqdm.auto import tqdm
import requests
import pandas as pd
import geopandas as gpd
from docx import Document


tqdm.pandas()
client_cert = "20250129_IDU_ADGX_LeonTur.crt"
ca_cert = "onti-ca.crt"
client_key = "DECFILE"

In [3]:
def request_from_llm(prompt: str) -> str:
    """
    Function extracts prompt from llm api
    Args:
        prompt (str): prompt to execute
    Returns:
        str: request from llm
    """

    headers = {
        "Content-Type": "application/json"
    }
    data = {
        "model": "llama3.3",
        "prompt": prompt,
        "stream": False,
    }
    response = requests.post(
        url="http://a.dgx:11434/api/generate",
        headers=headers,
        data=json.dumps(data),
        cert=(client_cert, client_key),
        verify=ca_cert
    )
    if response.status_code == 200:
        return response.json()["response"]
    return ""

In [4]:
def construct_lemma_prompt(text: str) -> str:

    prompt = f"""
        Приведи все слова в тексте в первоначальную форму слова{text}
        Слова должны быть преведены в одну строчку через запятую, без пробелов и переходов на новую строчку
        """
    return prompt

def get_tags(text: str) -> str:

    prompt = construct_lemma_prompt(text)
    tags_from_llm = request_from_llm(prompt)
    return tags_from_llm

In [5]:
df = pd.read_csv("with_names.csv")

In [8]:
df["tags"] = df["text"].progress_apply(get_tags)

  0%|          | 0/1265 [00:00<?, ?it/s]

In [9]:
df["tags"]

0                           привести, слово, текст, форма
1                                  Закон,Санкт-Петербурга
2       Мне не было дано исходного текста для обработк...
3       Слово "социально-экономический" состоит из дву...
4                                развитие,Санкт-Петербург
                              ...                        
1260    улучшение,инвестиционный,климат,Санкт-Петербур...
1261    ретроспективный, данные, взаимосвязь, уровень,...
1262    Реализация,Стратегия,осуществляется,посредство...
1263    Государственная,программа,Санкт-Петербург,разр...
1264    Привести,государственный,программа,утверждать,...
Name: tags, Length: 1265, dtype: object